In [8]:
from joblib import load
import pandas as pd
import code.operators as ops
import statistics

In [2]:
lr_bias = load('lr_loan_bias.joblib')
lr = load('lr_loan.joblib')

In [4]:
mock = pd.read_csv('mock_input.csv')
print(mock)

    col_id  data_type      mean        std  min  max
0  column1     binary       NaN        NaN  NaN  NaN
1  column2     binary       NaN        NaN  NaN  NaN
2  column3     binary       NaN        NaN  NaN  NaN
3  column4     binary       NaN        NaN  NaN  NaN
4  column5     binary       NaN        NaN  NaN  NaN
5  column6     binary       NaN        NaN  NaN  NaN
6  column7  numerical  0.344633  22.782046  NaN  NaN
7  column8  numerical  0.019596  21.520722  NaN  NaN
8  column9  numerical -0.163414  22.534570  NaN  NaN


In [6]:
def create_output(input, model, disadvantaged, request_id):
    output = pd.DataFrame(columns=['request_id'])
    output = output.set_index('request_id')
    
    #calculating disparity index and appending to output:
    disparity = []
    #simulation of 100 times since data is randomly normal distributed
    for i in range(100):
        #generating synthetic data based on user input:
        X_new = ops.data_input(input)
        #create prediction for synthetic data with uploaded model:
        pred_n = ops.create_eval(model, X_new)
        #calculate disparity index
        disparity.append(ops.disparity_values(pred_n, 'Pred', 'column1', disadvantaged))
    output.loc[request_id, 'disparity_index'] = statistics.mean(disparity)

    mean_diff = []
    for i in range(100):
        X_new = ops.data_input(input)
        pred_n = ops.create_eval(model, X_new)
        #calculate mean difference
        mean_diff.append(ops.mean_diff_values(pred_n, 'Pred', 'column1', disadvantaged))
    output.loc[request_id, 'mean_difference'] = statistics.mean(mean_diff)
    
    equal_ops = []
    for i in range(100):
        X_new = ops.data_input(input)
        pred_n = ops.create_eval(model, X_new)
        #calculate equal opportunity index
        equal_ops.append(ops.equal_ops_values(X_new,pred_n, 'Pred', 'column1', disadvantaged))
    output.loc[request_id, 'equal_opportunity'] = statistics.mean(equal_ops)

    positive = []
    for i in range(100):
        X_new = ops.data_input(input)
        pred_n = ops.create_eval(model, X_new)
        #calculate all positive predictions
        positive.append(pred_n['Pred'].sum())
    output.loc[request_id, 'positive_outcomes'] = statistics.mean(positive)/1000
    
    return output

In [10]:
create_output(mock, lr, 0, 30)

,disparity_index,mean_difference,equal_opportunity,positive_outcomes
request_id,,,,
30,0.02948,0.02123,0.031085,0.751
